In [ ]:
filenames = [
    # 'O2 Grouper Pipeline - Task List.xlsx',
    # 'P2 v3 Authorization_DS manager - list of tasks.xlsx',
    'P2 v3 Replatforming Tasks List.xlsx',
    # 'P2 testing tasks.xlsx',
]


In [ ]:
import tools.dev_o2_excel_ingestor as o2

for filename in filenames:
    print("Ingesting: ", filename)
    rdf = o2.generate_rdf_from_xlsx('./tools/', filename)
    o2.send_rdf_to_dgraph('localhost:9080', rdf)
    print("Finished ingestion: ", filename)



In [1]:
def get_dfp_dfd():
    import tools.lib.pmtx_client.query_baselines as rb
    import requests
    import json
    import pandas as pd
    from time import time


    url = 'http://localhost:8080/graphql'


    query_project_filter = {
        "or": [
            {"name": {"allofterms": "P2 v3 Replatforming Tasks List.xlsx"}},
            {"name": {"allofterms": "O2 Grouper Pipeline - Task List.xlsx"}},
            {"name": {"allofterms": "P2 v3 Authorization_DS manager - list of tasks.xlsx"}}
        ]
    }


    normalized_baseline = rb.request_and_normalize_baselines_from_pmtx(url, query_project_filter)
    dfp, dfd = rb.baseline_to_pandas_df(normalized_baseline)



    query = '''
        query ($projects_ids:[ID!]) {
            queryProject (filter: {and: [{id: $projects_ids} ]}) {
                id
                customFields
            }
        }
    '''

    r = requests.post(url=url, json={"query": query, "variables": {"projects_ids": dfp.project_id.to_list()}})

    project_details = r.json()['data']['queryProject']

    for project in project_details:
        if project['customFields'] is not None:
            details = json.loads(project['customFields'])
            project['sprint'] = (int(details['sprint'].split(' ')[1]) if details['sprint'].split(' ')[1].isdigit() else None) if 'sprint' in details else None
        else: 
            project['sprint'] = None
        del project['customFields']

    project_details = pd.DataFrame(project_details)
    dfp = dfp.merge(project_details, how='left', left_on='project_id', right_on='id')
    dfp.drop(columns=['id'], inplace=True)

    print("acquiring completed.")

    return dfp, dfd


In [ ]:
import tools.lib.task_assignee_estimators.solver_base as base

est = base.SolverBase(dfp,dfd)

est.config()
est.initialize()

est.find_incorrect_dependencies_FS()

# est.lp

# dfp[dfp.start.notnull() & dfp.finish.notnull()]

In [ ]:
from time import time
import pandas as pd

import tools.lib.task_assignee_estimators.primitive_estimation as estimator




solver = estimator.ProposeAssigment()

start_date = dfp.start.min()


projects = dfp.copy(deep=True)
dependencies = dfd.copy(deep=True)

solver.av = estimator.create_availability_calendar(number_of_users=9)
solver.av['project_id'] = None

algo_time_start = time()

for sprint in [6,7,8,9,10]:
    solver.projects = projects[(projects.sprint < sprint)]
    # print(solver.projects[solver.projects.sprint == 6].shape[0])
    solver.dependencies = dependencies
    solver.av[solver.av.project_id.notnull()]

    # # Calculation part
    solver.initialize(start_date)
    solver.av[solver.av.project_id.notnull()]
    # x1 = solver.av.copy(deep=True)

    # finish_date = solver.assign_projects_from_last_task_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)
    finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

    solver.av[solver.av.project_id.notnull()]
    # x2 = solver.av.copy(deep=True)

    solver.update_projects()
    solver.av[solver.av.project_id.notnull()]


    for index, row in solver.projects.iterrows(): # TODO: find better way to update
        projects.loc[projects.project_id == row['project_id'], 'start'] = solver.projects[solver.projects.project_id == row['project_id']].start.iloc[0]
        projects.loc[projects.project_id == row['project_id'], 'finish'] = solver.projects[solver.projects.project_id == row['project_id']].finish.iloc[0]


    break

# ## resolve rest (NULLs)
# solver.projects = projects
# # print(solver.projects[solver.projects.sprint == 6].shape[0])
# solver.dependencies = dependencies

# # # Calculation part
# solver.initialize(start_date)

# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

# solver.update_projects()

# # projects[(projects.sprint < sprint), 'start'] = solver.projects
# for index, row in solver.projects.iterrows(): # TODO: find better way to update
#     projects.loc[projects.project_id == row['project_id'], 'start'] = solver.projects[solver.projects.project_id == row['project_id']].start.iloc[0]
#     projects.loc[projects.project_id == row['project_id'], 'finish'] = solver.projects[solver.projects.project_id == row['project_id']].finish.iloc[0]


algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")




In [ ]:
sprt = 6
print("min: ", solver.projects[solver.projects.sprint == sprt].start.min())
print("min: ", solver.projects[solver.projects.sprint == sprt].start.max())

In [2]:
import tools.lib.pmtx_client.mutate_baselines as mb
import datetime

url = 'http://localhost:8080/graphql'

solver.projects.drop(columns=['sprint'], inplace=True)

root_id = "0x2e8a"
baseline_name = "X last finish solver - from actual times " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


baseline_id = mb.add_baseline_and_return_id(url, baseline_name, root_id)
mb.add_project_baseline(url, solver.projects, baseline_id, root_id)
mb.modify_project_baseline_predecessors(url, solver.dependencies, baseline_id)
mb.add_resource_baseline(url, solver.av, baseline_id)

print("Finished.")

d:\pmt_x\tools\lib\pmtx_client\mutate_baselines.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  children['parent_id'] = root_id
{'data': {'addProjectBaseline': {'projectBaseline': [{'id': '0xa6dc', 'project': {'id': '0x2e8a'}, 'baseline': {'id': '0xa61d'}}]}}, 'extensions': {'touched_uids': 5724, 'tracing': {'version': 1, 'startTime': '2021-05-11T20:48:16.5740502Z', 'endTime': '2021-05-11T20:48:16.689654Z', 'duration': 115603800, 'execution': {'resolvers': [{'path': ['addProjectBaseline'], 'parentType': 'Mutation', 'fieldName': 'addProjectBaseline', 'returnType': 'AddProjectBaselinePayload', 'startOffset': 3398400, 'duration': 112166700, 'dgraph': [{'label': 'mutation', 'startOffset': 16456700, 'duration': 88326000}, {'label': 'query', 'star

In [ ]:
from time import time
import pandas as pd

import tools.lib.task_assignee_estimators.primitive_estimation as estimator






solver = estimator.ProposeAssigment()

start_date = dfp.start.min()

solver.projects = dfp.copy(deep=True)
solver.dependencies = dfd.copy(deep=True)
# solver.av = pd.DataFrame([], columns=['user_id', 'start', 'finish'])
solver.av = estimator.create_availability_calendar(number_of_users=9)
# solver.av = expander.create_availability_frame()
solver.av['project_id'] = None
print(solver.av.shape[0])

# Calculation part

solver.initialize(start_date)


algo_time_start = time()


finish_date = solver.assign_projects_infinite_resources()
# finish_date = solver.assign_projects_to_resources_first_free(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

print(solver.av.shape[0])
solver.update_projects()

algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
# print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")

In [1]:
def get_dfp_dfd():
    import tools.lib.pmtx_client.query_baselines as rb
    import requests
    import json
    import pandas as pd
    from time import time


    url = 'http://localhost:8080/graphql'


    query_project_filter = {
        "or": [
            {"name": {"allofterms": "P2 v3 Replatforming Tasks List.xlsx"}},
            {"name": {"allofterms": "O2 Grouper Pipeline - Task List.xlsx"}},
            {"name": {"allofterms": "P2 v3 Authorization_DS manager - list of tasks.xlsx"}}
        ]
    }


    normalized_baseline = rb.request_and_normalize_baselines_from_pmtx(url, query_project_filter)
    dfp, dfd = rb.baseline_to_pandas_df(normalized_baseline)



    query = '''
        query ($projects_ids:[ID!]) {
            queryProject (filter: {and: [{id: $projects_ids} ]}) {
                id
                customFields
            }
        }
    '''

    r = requests.post(url=url, json={"query": query, "variables": {"projects_ids": dfp.project_id.to_list()}})

    project_details = r.json()['data']['queryProject']

    for project in project_details:
        if project['customFields'] is not None:
            details = json.loads(project['customFields'])
            project['sprint'] = (int(details['sprint'].split(' ')[1]) if details['sprint'].split(' ')[1].isdigit() else None) if 'sprint' in details else None
        else: 
            project['sprint'] = None
        del project['customFields']

    project_details = pd.DataFrame(project_details)
    dfp = dfp.merge(project_details, how='left', left_on='project_id', right_on='id')
    dfp.drop(columns=['id'], inplace=True)

    print("acquiring completed.")

    return dfp, dfd















from time import time
import pandas as pd

import tools.lib.task_assignee_estimators.last_finish_solver as estimator


dfp, dfd = get_dfp_dfd()

solver = estimator.LastFinishSolver(dfp.copy(deep=True),dfd.copy(deep=True))

solver.config()


solver.av = estimator.LastFinishSolver.create_availability_calendar(start_time=pd.Timestamp('2021-05-02 21:04:56.092887+0000'), number_of_users=5)

solver.av['project_id'] = None
print(solver.av.shape[0])

# Calculation part

solver.initialize()


algo_time_start = time()


finish_date = solver.allocate_projects(create_plan_from_scratch=False)
# finish_date = solver.assign_projects_to_resources_first_free(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

print(solver.av.shape[0])
solver.update_projects()

algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
# print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")

acquiring completed.
1075
1146
Project finish timestamp: None
Calculation time [s]: 6.133681058883667
Finished.


In [2]:
av = solver.av.copy(deep=True)
av['worktime_grouped'] = av.finish - av.start
grouped = av[['project_id', 'worktime_grouped']].groupby('project_id').sum()
merged = grouped.merge(solver.projects[['project_id', 'worktime']], on='project_id')
print('Number of same values', merged[merged.worktime_grouped == merged.worktime].shape[0])
print('Number of different values', merged[merged.worktime_grouped != merged.worktime].shape[0])
print(merged.worktime_grouped.sum(), 'vs', merged.worktime.sum())
merged[merged.worktime_grouped != merged.worktime]

Number of same values 82
Number of different values 0
22 days 23:30:00 vs 22 days 23:30:00


,project_id,worktime_grouped,worktime


In [4]:
len(solver.av.project_id.unique())

39

In [5]:
solver.av[solver.av.project_id == '0x2edb']

,start,finish,resource_id,project_id
439,2021-05-14 08:00:00.092887+00:00,2021-05-14 15:30:00.092887+00:00,2,0x2edb
1119,2021-05-13 11:30:00.092887+00:00,2021-05-13 16:00:00.092887+00:00,2,0x2edb


In [6]:
solver.projects[solver.projects.project_id == '0x2edb']

,project_id,parent_id,start,finish,worktime
138,0x2edb,0x3105,2021-05-13 11:30:00.092887+00:00,2021-05-14 15:30:00.092887+00:00,0 days 12:00:00
